In [ ]:
#import dependencies
import pandas as pd
import gmaps
import requests

#import api key
from config import g_key

In [ ]:
#load cities.csv file
file_to_load = "weather_data/cities.csv"
city_data_df = pd.read_csv(file_to_load)

#configure gmaps to use api key
gmaps.configure(api_key=g_key)

city_data_df.dtypes

In [ ]:
#creating variables
revised_city_data = city_data_df.loc[city_data_df["Max Temp"] > 0]
locations = revised_city_data[["Lat", "Lng"]]
max_temp = revised_city_data["Max Temp"]

#creating heatmap for temperature
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)


fig


In [ ]:
#creating variables
locations = revised_city_data[["Lat", "Lng"]]
humidity = revised_city_data["Humidity"]
#creating heatmap for Humidity
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

In [ ]:
#creating variables
locations = revised_city_data[["Lat", "Lng"]]
clouds = revised_city_data["Cloudiness"]
#creating heatmap for cloudiness
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

In [ ]:
#creating variables
locations = revised_city_data[["Lat", "Lng"]]
wind = revised_city_data["Wind Speed"]
#creating heatmap for cloudiness
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

In [ ]:
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [ ]:
#identifying preferred cities based on user input
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.count()

In [ ]:
#creating hotel_df 
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}


In [ ]:
 info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
    

In [ ]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [ ]:
hotel_df.head()

In [ ]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,\
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig